Notebook that analyses annotator agreement - still need to tidy up and add some comments to this notebook, or to delete it as I'm not sure we will make use of it.

In [ ]:
import os
import pandas as pd
from mtqe.data.loaders import load_ced_test_data, load_ced_data
from mtqe.utils.paths import EVAL_DIR, PREDICTIONS_DIR
from mtqe.utils.plots import create_annotator_cm_plot
from mtqe.utils.language_pairs import LI_LANGUAGE_PAIRS_WMT_21_CED
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve

In [ ]:
EXPERIMENT_GROUP_NAME = 'train_monolingual_auth_data'
DATA_SPLIT = 'dev'
THRESHOLD_STRATEGY = 'default'
FILE_SUFFIX = 'median_results.csv'
CHART_NAME = 'Monolingual Authentic Data'

In [ ]:
eval_files = [os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME, file) for file in os.listdir(os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME)) if file.endswith(FILE_SUFFIX)]
enja_separate = False
if os.path.isdir(os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME + '_enja')):
    eval_files.extend([os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME + '_enja', file) for file in os.listdir(os.path.join(EVAL_DIR, EXPERIMENT_GROUP_NAME + '_enja')) if file.endswith(FILE_SUFFIX)])
    enja_separate = True
    

In [ ]:
dfs = []
for file in eval_files:
    df = pd.read_csv(file)
    dfs.append(df)

df_meta_results = pd.concat(dfs)

In [ ]:
df_meta_results = df_meta_results[(df_meta_results['threshold_strategy'] == THRESHOLD_STRATEGY) & (df_meta_results['split'] == DATA_SPLIT)]

In [ ]:
df_meta_results.head()

In [ ]:
# Get lists of data to create plot
preds = []
targets = []
indices = []
titles = []

for lp in LI_LANGUAGE_PAIRS_WMT_21_CED:
    seed = df_meta_results[df_meta_results['language_pair']==lp].iloc[0]['seed']
    mcc = df_meta_results[df_meta_results['language_pair']==lp].iloc[0]['MCC']
    precision = df_meta_results[df_meta_results['language_pair']==lp].iloc[0]['precision']
    recall = df_meta_results[df_meta_results['language_pair']==lp].iloc[0]['recall']
    titles.append('Language pair:' + lp)# + '\nMCC:' + str(mcc.round(3)) + '\nPrecision:' + str(precision.round(3)) + '\nRecall:' + str(recall.round(3)))
    threshold = df_meta_results[df_meta_results['language_pair']==lp].iloc[0]['threshold']
    file_prefix = lp + '_' + DATA_SPLIT + '_' + str(seed)
    if lp == 'en-ja' and enja_separate:
        folder = EXPERIMENT_GROUP_NAME + '_enja'
    else:
        folder = EXPERIMENT_GROUP_NAME
    for file in os.listdir(os.path.join(PREDICTIONS_DIR, 'ced_data', folder)):
        if file.startswith(file_prefix) and file.endswith('.csv'):
            # found predictions
            df_preds = pd.read_csv(os.path.join(PREDICTIONS_DIR, 'ced_data', folder, file))
            scores = df_preds['score'].to_numpy()
            scores = 1 - scores
            binary_scores = scores > threshold
            binary_scores = binary_scores.astype('int')
            df_preds['binary_score'] = binary_scores
            # preds.append(binary_scores)
            preds.append(df_preds)
    if DATA_SPLIT == 'test':
        targets.append(1 - load_ced_test_data(lp)['score'].to_numpy())
    else:
        df = load_ced_data(DATA_SPLIT, lp, incl_annotations=True)
        df['annotations'] = df['annotations'].str.replace('[', '')
        df['annotations'] = df['annotations'].str.replace(']', '')
        df[['annotator_1', 'annotator_2', 'annotator_3']] = df['annotations'].str.split(',', expand=True).astype('int')
        df['total_score']= df['annotator_1'] + df['annotator_2'] + df['annotator_3']
        # targets.append(df['total_score'])
        targets.append(df)
        # targets.append(df['annotations'])
        indices.append(load_ced_data(DATA_SPLIT, lp)['idx'])

In [ ]:
preds_targets = []
preds_cleaned = []
targets_cleaned = []

for ind, _ in enumerate(preds):
    df = pd.concat([preds[ind], targets[ind]], axis=1)
    print('shape before:', df.shape)
    df = df[(df['annotator_1'].isin([0,1])) & (df['annotator_2'].isin([0,1])) & (df['annotator_3'].isin([0,1]))]
    print('shape after: ', df.shape)
    preds_targets.append(df)
    preds_cleaned.append(df['binary_score'])
    targets_cleaned.append(df['total_score'])

In [ ]:
fig = create_annotator_cm_plot(CHART_NAME + ' - ' + DATA_SPLIT + ' - Annotator scores vs predictions', LI_LANGUAGE_PAIRS_WMT_21_CED, preds_cleaned, targets_cleaned)